## Importamos las librerías

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from modelo_energetico.col_reductor import reduce_columns_sum, reduce_columns_avg, reduce_columns_period_sum, reduce_columns_period_avg, total_q_hour 
from modelo_energetico.nn_gen import init_vanilla_nn
from modelo_energetico.scaler import MultiScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping

2021-10-19 17:38:26.628503: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-10-19 17:38:26.628566: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Cargamos los datasets

In [2]:
X = pd.read_csv('../raw_data/X.csv', index_col = 0)
y = pd.read_csv('../raw_data/y.csv', index_col = 0)

print(X.shape)
print(y.shape)

(7500, 12115)
(7500, 5376)


In [10]:
y.head()

,Q_AC_OFFICE_0,Q_AC_OFFICE_1,Q_AC_OFFICE_2,Q_AC_OFFICE_3,Q_AC_OFFICE_4,Q_AC_OFFICE_5,Q_AC_OFFICE_6,Q_AC_OFFICE_7,Q_AC_OFFICE_8,Q_AC_OFFICE_9,...,T_INT_OFFICE_662,T_INT_OFFICE_663,T_INT_OFFICE_664,T_INT_OFFICE_665,T_INT_OFFICE_666,T_INT_OFFICE_667,T_INT_OFFICE_668,T_INT_OFFICE_669,T_INT_OFFICE_670,T_INT_OFFICE_671
index,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,23.0000,23.0000,23.0000,23.0000,23.7004,24.7010,25.3532,25.8526,26.0000,26.0000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20.8157,20.9522,21.0344,21.0586,21.0659,21.0449,20.9765,20.8875,20.6578,20.3838
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,23.5000,23.5000,23.5000,23.5000,23.5000,23.5000,23.5018,23.5032,23.5006,23.4927
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,22.1434,22.2447,22.3606,22.4758,22.5641,22.5912,22.5744,22.5523,22.5276,22.5080
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,23.5000,23.5000,23.5000,23.5000,23.9176,24.7056,25.2880,25.5000,25.5000,25.5000


In [12]:
columns_y = [x[:-4] for x in y.columns if '671' in x]
columns_y

['Q_AC_OFFICE',
 'Q_HEAT_OFFICE',
 'Q_PEOPLE',
 'Q_EQP',
 'Q_LIGHT',
 'Q_AHU_C',
 'Q_AHU_H',
 'T_INT_OFFICE']

In [13]:
y_tmp = y.head()

y_per_h = None
for i in range(y_tmp.shape[0]):
    new_data_dict = {}
    for col in columns_y:
        columns_y_j = [f'{col}_{j}' for j in range(672)]
        new_data_dict[col] = y_tmp[columns_y_j].iloc[[i]].values[0].T
    new_df = pd.DataFrame(data = new_data_dict)
  
    if not(y_per_h is None):
        y_per_h = pd.concat([y_per_h, new_df])
        print(f'Se agregaron {i*672} filas de {y_tmp.shape[0]*672}')
    else:
        y_per_h = new_df
y_per_h

Se agregaron 672 filas de 3360
Se agregaron 1344 filas de 3360
Se agregaron 2016 filas de 3360
Se agregaron 2688 filas de 3360


,Q_AC_OFFICE,Q_HEAT_OFFICE,Q_PEOPLE,Q_EQP,Q_LIGHT,Q_AHU_C,Q_AHU_H,T_INT_OFFICE
0,0.0000,0.0,0.0,0.0000,0.0000,0.0,0.0,23.0000
1,0.0000,0.0,0.0,1.4718,0.2100,0.0,0.0,22.9364
2,0.0000,0.0,0.0,1.4718,0.2100,0.0,0.0,22.8072
3,0.0000,0.0,0.0,1.4718,0.2100,0.0,0.0,22.6742
4,0.0000,0.0,0.0,1.4718,0.2100,0.0,0.0,22.5406
...,...,...,...,...,...,...,...,...
667,0.0000,0.0,0.0,6.3142,0.1669,0.0,0.0,24.7056
668,7.0401,0.0,0.0,6.3142,0.1669,0.0,0.0,25.2880
669,24.4660,0.0,0.0,6.3142,0.1669,0.0,0.0,25.5000
670,19.2376,0.0,0.0,6.3142,0.1669,0.0,0.0,25.5000


In [ ]:
y.columns

In [ ]:
cols_t = y.iloc[: , -672:]
X = pd.concat([X, cols_t], axis = 1)
y = y.iloc[: , :-672]

## Graficamos los features

In [14]:
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
t_int_cols = [f'T_INT_OFFICE_{j}' for j in range(672)]
y[t_int_cols].iloc[[0]].values[0]
# sns.scatterplot(y[t_int_cols].iloc[[0]].values[0])

array([23.    , 22.9364, 22.8072, 22.6742, 22.5406, 22.4144, 22.312 ,
       22.578 , 23.1005, 23.5308, 23.8737, 24.    , 24.    , 24.    ,
       24.    , 24.    , 24.    , 24.    , 24.    , 24.    , 24.1127,
       24.2478, 24.2683, 24.2496, 24.2054, 24.1468, 24.079 , 23.992 ,
       23.8776, 23.7475, 23.6272, 23.7865, 24.    , 24.    , 24.    ,
       24.    , 24.    , 24.    , 24.    , 24.    , 24.    , 24.    ,
       24.3074, 24.6512, 24.647 , 24.5043, 24.2947, 24.0809, 23.8711,
       23.6677, 23.4736, 23.2911, 23.1149, 22.9401, 22.7993, 23.0319,
       23.1591, 23.    , 23.    , 23.    , 23.    , 23.    , 23.    ,
       23.    , 23.    , 23.    , 23.    , 23.369 , 23.91  , 24.1347,
       24.203 , 24.2141, 24.1902, 24.1471, 24.0911, 24.0265, 23.9552,
       23.8728, 23.8024, 23.8616, 23.9739, 24.    , 24.    , 24.    ,
       24.    , 24.    , 24.    , 24.    , 24.    , 24.    , 24.237 ,
       24.5564, 24.6856, 24.7007, 24.6199, 24.518 , 24.4034, 24.2727,
       24.1347, 23.9

## Vemos que features son series

In [ ]:
columns_s = [x[:-4] for x in X.columns if '671' in x]
columns_s

## Armamos datasets reducidos

In [ ]:
columns_c = X.columns[0:19]
columns_c

In [ ]:
columns_c = X.columns[0:16]

In [ ]:
y = total_q_hour(y)
y.head()

In [ ]:
X_reduced = {}
y_reduced = {}
for div in [1, 2, 4, 7, 14, 28]:
    X_reduced[div] = pd.concat([X[columns_c], reduce_columns_period_avg(X, columns_s, div)], axis = 1)
    y_reduced[div] = reduce_columns_period_sum(y, ['Q'], div)

In [ ]:
X_reduced[28].head()

In [ ]:
y_reduced[28].head()

## Probamos SGD con distintos períodos

In [ ]:
pipe = Pipeline([
          ("scaling" , MultiScaler(scaler = "RobustScaler")),
        ("model", MultiOutputRegressor(estimator = SGDRegressor()))
])

In [ ]:
for i in pipe.get_params().keys():
    print(i)

In [ ]:
params = {
    'scaling__scaler' : [StandardScaler(), RobustScaler(), MinMaxScaler() ],     
    'model__estimator__loss': ['huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
    'model__estimator__alpha': [1, 0.1, 0.01, 0.001, 0.0001],
    'model__estimator__l1_ratio': [1, 0.1, 0.01, 0.001, 0.0001],
}

grid = GridSearchCV(pipe, 
                    param_grid= params, 
                    cv=5,
                    n_jobs=-1,
                    verbose=1,
                    scoring = 'r2'
                   )

In [ ]:
models = {}
results = {}
X_reduced_train = {}
X_reduced_test = {}
y_reduced_train = {}
y_reduced_test = {}

In [ ]:
for div in [7, 14, 28]:
    X_reduced_train[div], X_reduced_test[div], y_reduced_train[div], y_reduced_test[div] = train_test_split(
        X_reduced[div], y_reduced[div], test_size=0.3, random_state=42)

In [ ]:
for div in [7, 14, 28]:
    grid.fit(X_reduced_train[div], y_reduced_train[div])
    
    start = datetime.now()
    best_estimator = grid.best_estimator_.fit(X_reduced_train[div], y_reduced_train[div])
    stop = datetime.now()
    best_score = best_estimator.score(X_reduced_test[div], y_reduced_test[div])
    time = (stop - start).seconds
  
    models[div] = best_estimator
    results[div] = (best_score, time)

In [ ]:
results

In [ ]:
models[7].named_steps.scaling

In [ ]:
models[7].named_steps.model.estimator

## Probamos redes neuronales

In [ ]:
nn_models = {}
nn_results = {}
nn_histories = {}

networks = [(10, 10, 5), (15, 10, 10, 5)] ## Probar distintas combinaciones
batch_size = [60, 80, 100] ## Probar distintas combinaciones
epochs = [10, 20] ## Probar distintas combinaciones
param_grid = dict(batch_size=batch_size, epochs=epochs)

es = EarlyStopping(patience=25, restore_best_weights=True)

for div in [7, 14, 28]:   
    input_dim = X_reduced_train[div].shape[1]
    output_dim = y_reduced_train[div].shape[1]

    for i, network in enumerate(networks):
    
    grid = GridSearchCV(estimator=KerasRegressor(build_fn=init_vanilla_nn, 
                                                 n_neurons=network,  
                                                 input_dim=input_dim, 
                                                 output_dim=output_dim), 
                        param_grid=param_grid, n_jobs=-1, cv=3)
    
    grid.fit(X_reduced_train[div], y_reduced_train[div], verbose=1)
    
    best_model = grid.best_estimator_

    start = datetime.now()
    hist = best_model.fit(X_reduced_train[div], y_reduced_train[div])
    stop = datetime.now()
    time = (stop - start).seconds
    best_score = max(hist.history['mae'])

    nn_results[(div, i)] = (best_score, time)
    nn_histories[(div, i)] = hist

In [ ]:
nn_results[(14, 1)]